In [1]:
import nengo
import nengo_spa as spa
import pytry
import numpy as np
from random import shuffle
import random
import sys, os
import math
from IPython import display
from scipy.optimize import brute, minimize
from scipy.stats import sem
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.style as style

import os

from itertools import product



# Import our classes
sys.path.append('..')
import experiments as xps
from experiments import create_xp
from model import ExperimentRun, create_vocab
from data import Data

# Optimization and model parameters

In [2]:
D = int(96)  # the dimensionality of the vectors
PROC_FDBCK = .9
PROC_FDBCK_SYN = .005
GW_FDBCK = 1
GW_SCALE = 20 # scale the input of GW for saturation and catching very short visual stimuli
BG_THR = .1
BG_BIAS = .5
STIM_DURATION = .029
FIX_DURATION = .5
N_NEURONS_SCALE = 1
N_NEURONS_SCALE_COMBINED = .25
INTEGRATOR_RESET = False
N_SAMPLES = 10000

N_BLOCKS_PER_OPERATION = 1 # default: 10
N_TRIALS_PER_DIGIT = 2 # default: 5
N_DIFFERENT_DIGITS = 4 # default: 4
N_DIFFERENT_OPERATIONS = 3 # default: 3

data_dir = "evaluate_fit/fit_no_threshold"


In [3]:
t_sensos = [0]#np.linspace(.05, .2, 5)
s_evids = np.linspace(5, 15, 3)
crosstalks = np.linspace(.05, .25, 5)
print(s_evids)
print(crosstalks)

for s in range(1000):
    STARTING_SEED = 1+s
    N_SEEDS = 1

    rmses = np.zeros((len(t_sensos), len(s_evids), len(crosstalks)))

    i=0
    for condition in product(*tuple([range(len(param_values)) for param_values in [t_sensos, s_evids, crosstalks]])):
        i+=1
        print('condition ', i, '/', rmses.size, ':', condition)  
        print('\t sensory delay:', t_sensos[condition[0]]*1000)
        print('\t evidence strength:', s_evids[condition[1]])
        print('\t crosstalk:', crosstalks[condition[2]])
        data = []

        if True:

            print('\t Simulating...')
            for i,seed in enumerate(range(STARTING_SEED, STARTING_SEED+N_SEEDS)):
                print('\t seed '+str(i+1)+'/'+str(N_SEEDS))
                data_filename = 'run_seed'+str(seed)+'_sevid'+str(s_evids[condition[1]])+'_crosstalk'+str(crosstalks[condition[2]])         

                try:
                    seed_data = pd.read_pickle(data_dir+'/'+data_filename+'.csv')

                except FileNotFoundError:
                    xp = create_xp(N_BLOCKS_PER_OPERATION, N_TRIALS_PER_DIGIT, N_DIFFERENT_DIGITS, N_DIFFERENT_OPERATIONS, STIM_DURATION, FIX_DURATION, seed)
                    results = ExperimentRun().run(
                        n_neurons_scale=N_NEURONS_SCALE,
                        n_neurons_scale_combined=N_NEURONS_SCALE_COMBINED,
                        s_evidence=s_evids[condition[1]],
                        s_crosstalk=crosstalks[condition[2]],
                        n_samples=N_SAMPLES,
                        vocab=create_vocab(D, seed),
                        xp=xp,
                        integrator_reset=True,
                        proc_feedback=PROC_FDBCK,
                        proc_feedback_synapse=PROC_FDBCK_SYN,
                        GW_feedback=GW_FDBCK,
                        GW_scale=GW_SCALE,
                        BG_thr=BG_THR,
                        BG_bias=BG_BIAS,
                        seed=seed,
                        backend="nengo_ocl",
                        data_dir=data_dir,
                        data_filename=data_filename,
                        plt=False
                    )
                    seed_data = results['data'].df
                    seed_data.to_pickle(data_dir+'/'+data_filename+'.csv')

                data.append(seed_data)


            data = Data(pd.concat(data))
            data.df['rt'] += t_sensos[condition[0]]*1000 # add sensory delay to reaction times

            rmse_fitness_error = data.rmse_fitness_error(compare_errorrates=False, compare_RTs=True, tasks=range(N_DIFFERENT_OPERATIONS))
            print('\t rmse:', rmse_fitness_error[0])
            print('\t error rate:', data.error_rate)
#             data.plot_fig2_simple(plot_humans=True)
#             data.plot_fig2_chained(plot_humans=True)

            rmses[condition] = rmse_fitness_error[0]





[ 5. 10. 15.]
[0.05 0.1  0.15 0.2  0.25]
condition  1 / 15 : (0, 0, 0)
	 sensory delay: 0
	 evidence strength: 5.0
	 crosstalk: 0.05
	 Simulating...
	 seed 1/1
	 rmse: 212.36945898979144
	 error rate: 0.33333333333333337
condition  1 / 15 : (0, 0, 1)
	 sensory delay: 0
	 evidence strength: 5.0
	 crosstalk: 0.1
	 Simulating...
	 seed 1/1
	 rmse: 206.032828112983
	 error rate: 0.33333333333333337
condition  1 / 15 : (0, 0, 2)
	 sensory delay: 0
	 evidence strength: 5.0
	 crosstalk: 0.15000000000000002
	 Simulating...
	 seed 1/1
	 rmse: 209.2211010811281
	 error rate: 0.33333333333333337
condition  1 / 15 : (0, 0, 3)
	 sensory delay: 0
	 evidence strength: 5.0
	 crosstalk: 0.2
	 Simulating...
	 seed 1/1
	 rmse: 208.5910312575315
	 error rate: 0.25
condition  1 / 15 : (0, 0, 4)
	 sensory delay: 0
	 evidence strength: 5.0
	 crosstalk: 0.25
	 Simulating...
	 seed 1/1
	 rmse: 189.88361394192205
	 error rate: 0.33333333333333337
condition  1 / 15 : (0, 1, 0)
	 sensory delay: 0
	 evidence stren

	 rmse: 130.8509435540108
	 error rate: 0.41666666666666663
condition  1 / 15 : (0, 0, 2)
	 sensory delay: 0
	 evidence strength: 5.0
	 crosstalk: 0.15000000000000002
	 Simulating...
	 seed 1/1
	 rmse: 143.7881890015392
	 error rate: 0.41666666666666663
condition  1 / 15 : (0, 0, 3)
	 sensory delay: 0
	 evidence strength: 5.0
	 crosstalk: 0.2
	 Simulating...
	 seed 1/1
	 rmse: 150.9594053450848
	 error rate: 0.41666666666666663
condition  1 / 15 : (0, 0, 4)
	 sensory delay: 0
	 evidence strength: 5.0
	 crosstalk: 0.25
	 Simulating...
	 seed 1/1
	 rmse: 173.97496548157952
	 error rate: 0.41666666666666663
condition  1 / 15 : (0, 1, 0)
	 sensory delay: 0
	 evidence strength: 10.0
	 crosstalk: 0.05
	 Simulating...
	 seed 1/1
	 rmse: 119.98120652173482
	 error rate: 0.16666666666666663
condition  1 / 15 : (0, 1, 1)
	 sensory delay: 0
	 evidence strength: 10.0
	 crosstalk: 0.1
	 Simulating...
	 seed 1/1
	 rmse: 93.49727401224489
	 error rate: 0.25
condition  1 / 15 : (0, 1, 2)
	 sensory del

	 rmse: 165.9485221407477
	 error rate: 0.41666666666666663
condition  1 / 15 : (0, 1, 0)
	 sensory delay: 0
	 evidence strength: 10.0
	 crosstalk: 0.05
	 Simulating...
	 seed 1/1
	 rmse: 83.19009815941652
	 error rate: 0.0
condition  1 / 15 : (0, 1, 1)
	 sensory delay: 0
	 evidence strength: 10.0
	 crosstalk: 0.1
	 Simulating...
	 seed 1/1
	 rmse: 84.32812321510389
	 error rate: 0.04166666666666663
condition  1 / 15 : (0, 1, 2)
	 sensory delay: 0
	 evidence strength: 10.0
	 crosstalk: 0.15000000000000002
	 Simulating...
	 seed 1/1
	 rmse: 106.93756559168781
	 error rate: 0.04166666666666663
condition  1 / 15 : (0, 1, 3)
	 sensory delay: 0
	 evidence strength: 10.0
	 crosstalk: 0.2
	 Simulating...
	 seed 1/1
	 rmse: 132.3974716589456
	 error rate: 0.04166666666666663
condition  1 / 15 : (0, 1, 4)
	 sensory delay: 0
	 evidence strength: 10.0
	 crosstalk: 0.25
	 Simulating...
	 seed 1/1
	 rmse: 161.11427361909168
	 error rate: 0.08333333333333337
condition  1 / 15 : (0, 2, 0)
	 sensory d

	 rmse: 129.02117910056927
	 error rate: 0.125
condition  1 / 15 : (0, 1, 3)
	 sensory delay: 0
	 evidence strength: 10.0
	 crosstalk: 0.2
	 Simulating...
	 seed 1/1
	 rmse: 140.5057380662096
	 error rate: 0.16666666666666663
condition  1 / 15 : (0, 1, 4)
	 sensory delay: 0
	 evidence strength: 10.0
	 crosstalk: 0.25
	 Simulating...
	 seed 1/1
	 rmse: 177.8563598665596
	 error rate: 0.16666666666666663
condition  1 / 15 : (0, 2, 0)
	 sensory delay: 0
	 evidence strength: 15.0
	 crosstalk: 0.05
	 Simulating...
	 seed 1/1
	 rmse: 97.51388035778214
	 error rate: 0.125
condition  1 / 15 : (0, 2, 1)
	 sensory delay: 0
	 evidence strength: 15.0
	 crosstalk: 0.1
	 Simulating...
	 seed 1/1
	 rmse: 98.67276251145445
	 error rate: 0.08333333333333337
condition  1 / 15 : (0, 2, 2)
	 sensory delay: 0
	 evidence strength: 15.0
	 crosstalk: 0.15000000000000002
	 Simulating...
	 seed 1/1
	 rmse: 113.09576529340545
	 error rate: 0.125
condition  1 / 15 : (0, 2, 3)
	 sensory delay: 0
	 evidence strengt

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

...

seed = 13
backend = 'nengo_ocl'
dt = 0.001
neuron_type = 'default'
n_neurons_scale = 1
n_neurons_scale_combined = 0.25
s_crosstalk = 0.15000000000000002
s_evidence = 15.0
n_samples = 10000
t_senso = 0
vocab = <nengo_spa.vocabulary.Vocabulary object at 0x7f9815573f50>
xp = <experiments.Xp1 object at 0x7f98167ef510>
proc_feedback = 0.9
proc_feedback_synapse = 0.005
GW_feedback = 1
GW_threshold = 0.5
GW_scale = 20
BG_thr = 0.1
BG_bias = 0.5
integrator_reset = True
data = <data.Data object at 0x7f98111856d0>
	 rmse: 112.87764360900718
	 error rate: 0.0
condition  1 / 15 : (0, 2, 3)
	 sensory delay: 0
	 evidence strength: 15.0
	 crosstalk: 0.2
	 Simulating...
	 seed 1/1
running run_seed13_sevid15.0_crosstalk0.2


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

Traceback (most recent call last):
  File "<ipython-input-3-a07e2be7505e>", line 30, in <module>
    seed_data = pd.read_pickle(data_dir+'/'+data_filename+'.csv')
  File "/home/ubuntu/anaconda3/envs/CTN/lib/python3.7/site-packages/pandas/io/pickle.py", line 170, in read_pickle
    f, fh = get_handle(fp_or_buf, "rb", compression=compression, is_text=False)
  File "/home/ubuntu/anaconda3/envs/CTN/lib/python3.7/site-packages/pandas/io/common.py", line 461, in get_handle
    f = open(path_or_buf, mode)
FileNotFoundError: [Errno 2] No such file or directory: 'evaluate_fit/fit_no_threshold/run_seed13_sevid15.0_crosstalk0.2.csv'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/CTN/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-a07e2be7505e>", line 53, in <module>
    plt=False
  File "/

TypeError: can only concatenate str (not "list") to str